In [1]:
# (Optional) Set the environment variable 'HF_ENDPOINT' to use the specified endpoint URL.
import os
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

In [2]:
from utils import chatgpt,MultiTQ,calculate_accuracy
from prompt import DECISION_INSTRUCTION,EXAMPLES,EXAMPLES_WITH_RATIONALS
import tqdm
import time
from multitq import get_actions,Inference
import pickle

multitq = MultiTQ()
inference = Inference(multitq.kg,multitq.id2text,multitq.rel2text)

# Start from new

In [3]:
inference = Inference(multitq.kg,multitq.id2text,multitq.rel2text,history=[],n_clusters = 10)

# or Load history_dict / inference

In [4]:
# # start from history_dict (use method instruct)
# with open('./history_dict_sample.pkl','rb') as f:
#     history_dict = pickle.load(f)
# inference = Inference(multitq.kg,multitq.id2text,multitq.rel2text,history=history_dict,n_clusters = 10)


In [5]:
## start from the saved inference
# with open('xxx.pkl','rb') as f:
#     inference = pickle.load(f)


# Inference Step

In [6]:
def decision(qid):
    inference.reset()
    attemp = 0
    entities = []
    max_attemp = 5
    q,head_text,rel_text,tail_text,time,event_text,answers = multitq[qid]
    print('Question {}:'.format(qid),q)
    hit = False
    while True:
        attemp+=1
        choose = get_actions(head_text,rel_text,tail_text,time,event_text,entities,inference.LLM_rel_choose)
        # print('All generated actions:',len(choose))
        # choose = inference.action_filter(choose)
        if len(choose) == 0:
            print('Unable to get answer!')
            inference.history.append([['Result: Unable to get answer!']])
            break
        print('-'*100)
        print('Step:',inference.tick)
        # print('Filtered generated actions:',len(choose))
        choose_text = '\n'.join(choose)
        # print(choose_text)

        history = ''
        for i in inference.history:
            history+= '\n'+ ' '.join(i)
        prompt = DECISION_INSTRUCTION.replace('{question}',q)
        #prompt = prompt.replace('{examples}',EXAMPLES_WITH_RATIONALS)
        prompt = prompt.replace('{actions}',choose_text)
        prompt = prompt.replace('{history}',history if len(history)>0 else 'None')
        # use method instruct
        # prompt = prompt.replace('{methodology}',inference.memory.get_method_instruction(q,head_text,tail_text))
        append = ' What should we do first?' if history == '' else ' What should we do next?'
        print('-'*100)
        # print("History Decision:",history)

        print("LLM's Decision:")
        response = chatgpt(prompt)
        print(response)
        kg_res,entities,end = inference.take_action(response)
            
        if end:
            print('The predicted answer is:',kg_res[0])            
            ans = multitq.question[qid]['answers']
            hit = inference.check_correctness(ans,kg_res[0])

            if hit:
                print('Correct!')
                inference.history.append([['Result: Correct!']])
                break
            else:
                print('Wrong!')
                inference.history.append([['Result: Wrong!']])
                break
                
        if attemp>=max_attemp:
            print('Unable to get answer!')
            inference.history.append([['Result: Unable to get answer!']])
            break
    his = inference.get_history()
    inference.reset()
    return hit,his

In [7]:
history_dict = {}
start_id = 10000
end_id = 10005
for qid in tqdm.tqdm(range(start_id,end_id)):
    hit,his = decision(qid)
    history_dict[qid] = {'question':multitq.question[qid],'process':his,'result':hit}
    # time.sleep(2)


  0%|          | 0/5 [00:00<?, ?it/s]

Question 10000: Who last wanted to negotiate with Japan before Vuk Jeremić?
----------------------------------------------------------------------------------------------------
Step: 0
----------------------------------------------------------------------------------------------------
LLM's Decision:
Action:
$get_tail_entity(Japan,Engage_in_negotiation,no time constraints)$

Reason:
To find out who last wanted to negotiate with Japan before Vuk Jeremić, I will retrieve the tail entity related to Japan's engagement in negotiation. This will help identify the individual or entity that expressed the intent to negotiate with Japan prior to Vuk Jeremić's involvement.
----------------------------------------------------------------------------------------------------
Step: 1
----------------------------------------------------------------------------------------------------
LLM's Decision:
Action:
$get_time(Vuk_Jeremić,Engage_in_negotiation,Japan)$

Reason:
To find out the specific time when

 20%|██        | 1/5 [00:13<00:54, 13.53s/it]

Action:
$get_head_entity(Vuk_Jeremić,Engage_in_negotiation,no time constraints)$

Reason:
Since we are looking for the last entity that wanted to negotiate with Japan before Vuk Jeremić, it is essential to identify who Vuk Jeremić engaged in negotiations with. By retrieving the head entity related to Vuk Jeremić's negotiations, we can find the relevant entities and their associated times, which will help us determine the last negotiation before him.
Unable to get answer!
Question 10001: Who visited France on 2 November 2011?
----------------------------------------------------------------------------------------------------
Step: 0
----------------------------------------------------------------------------------------------------
LLM's Decision:
Action:
$get_head_entity(France,Make_a_visit,2011-11-02)$

Reason:
I chose this action because the question specifically asks for who visited France on a particular date (2 November 2011). The relation "Make_a_visit" is relevant to identify th

 40%|████      | 2/5 [00:21<00:31, 10.55s/it]

...

Action:
$answer(2013)$

Reason:
The previous action retrieved the time when France condemned the Militia of France, which was on May 2, 2013. Since the question asks for the year, the correct answer is 2013.
The predicted answer is: 2013
Correct!
Question 10003: On 28 May 2005, which country announced its intention to cooperate with Japan?
----------------------------------------------------------------------------------------------------
Step: 0
----------------------------------------------------------------------------------------------------
LLM's Decision:
Action:
$get_tail_entity(Japan,Express_intent_to_engage_in_diplomatic_cooperation_(such_as_policy_support),2005-05-28)$

Reason:
This action is chosen because it directly queries the relationship of Japan expressing its intent to engage in diplomatic cooperation on the specified date of 28 May 2005. This should help identify the country that announced its intention to cooperate with Japan on that date.
--------------------------

 80%|████████  | 4/5 [00:28<00:05,  5.73s/it]

Action:
$answer(China)$

Reason:
The previous action successfully identified that on 28 May 2005, China announced its intention to cooperate with Japan. Therefore, the correct answer to the question is China, and I can provide it directly using the answer function.
The predicted answer is: China
Wrong!
Question 10004: After the Sudanese ministry, who was the first to express optimism about France?
----------------------------------------------------------------------------------------------------
Step: 0
----------------------------------------------------------------------------------------------------
LLM's Decision:
Action:
$get_tail_entity(Sudan,Make_optimistic_comment,no time constraints)$

Reason:
To find out who was the first to express optimism about France after the Sudanese ministry, I will query for the tail entity that corresponds to Sudan making an optimistic comment. This will help identify the individual or entity that expressed optimism following the Sudanese ministry's

100%|██████████| 5/5 [00:38<00:00,  7.61s/it]

Action:
$get_tail_entity(France,Make_optimistic_comment,no time constraints)$

Reason:
To find out who made optimistic comments about France after the Sudanese ministry, I need to identify the entities that made optimistic comments related to France. This will help me determine the first entity that expressed optimism about France after the specified time.
Unable to get answer!


# Save history_dict

In [8]:
with open('./history_dict.pkl','wb') as f:
    pickle.dump(history_dict,f)

# Evaluation

In [9]:
calculate_accuracy(history_dict)

before_last 	 0.0 1
equal 	 0.33 3
after_first 	 0.0 1
Overall Accuracy: 0.2


0.2